In [1]:
# Math
import numpy as np
import pandas as pd
from math import e

# Graphing
import matplotlib.pyplot as plt
import seaborn as sns

# Datetime converter
from datetime import datetime

In [2]:
# Read Datasets

# 1 Day Crypto Exchanges
exchanges = ['COINBASE', 'BINANCE', 'FTX', 'HUOBI', 'KUCOIN']
exchanges_cols = {1:['open_x', 'high_x', 'low_x', 'close_x', 'Basis_x', 'Volume_x', 'RSI_x', 'MACD_x', 'OnBalanceVolume_x'], 2:['open_y', 'high_y', 'low_y', 'close_y', 'Basis_y', 'Volume_y', 'RSI_y', 'MACD_y', 'OnBalanceVolume_y']}

# Open df
for exchange in exchanges:
    file_str_1D = 'datasets/{}_BTCUSDT, 1D.csv'.format(exchange)
    df = pd.read_csv(file_str_1D)
    
    # Remove irrelevant columns
    df.drop(['Signal', 'Volume MA', 'Histogram', 'Upper', 'Lower'], axis=1, inplace = True)
    
    if exchange == 'COINBASE':
        merge_df = df
        
    else:   
        if exchange != 'BINANCE':
            # Rename columns before merging to avoid autonaming
            keep_same = ['time']
            df.columns = ['{}{}'.format(c, '' if c in keep_same else '_y') for c in df.columns]
        
        # Merge df
        merge_df = pd.merge(merge_df, df, on = 'time', how = 'outer')
        
        # Take mean of columns, drop processed column and rename to avoid autonaming for next loop
        for i in range(len(exchanges_cols[1])):
            merge_df[exchanges_cols[1][i]+'y'] = merge_df[[exchanges_cols[1][i], exchanges_cols[2][i]]].mean(axis = 1)
            merge_df.drop([exchanges_cols[1][i], exchanges_cols[2][i]], axis = 1, inplace = True)
            merge_df = merge_df.rename(columns={exchanges_cols[1][i]+'y': exchanges_cols[1][i]})

            
others = ['CRYPTOCAP_BTC.D', 'CRYPTOCAP_ETH.D', 'CRYPTOCAP_TOTAL', 'CRYPTOCAP_TOTAL2', 'BCHAIN_MIREV']

# Looping and taking mean of other variables
for other in others:
    file_str_1D = 'datasets/{}, 1D.csv'.format(other)
    df = pd.read_csv(file_str_1D)[['time','open', 'close']]
    df[other + '_mean'] = df[['open', 'close']].mean(axis = 1)
    df = df[['time', other + '_mean']]
    # Merge with main df
    merge_df = pd.merge(merge_df, df, on = 'time', how = 'inner')

# Convert unix timestamp to datetime
merge_df['time'] = pd.to_datetime(merge_df['time'], unit = 's')
merge_df['time'] = pd.to_datetime(merge_df['time']).dt.date

merge_df = merge_df.rename(columns={'CRYPTOCAP_BTC.D_mean': 'btc_dom', 'CRYPTOCAP_ETH.D_mean': 'eth_dom', 'CRYPTOCAP_TOTAL_mean' : 'crypto_cap', 'CRYPTOCAP_TOTAL2_mean' : 'crypto_cap_exc_BTC', 'BCHAIN_MIREV_mean' : 'btc_reward'})

In [4]:
# Merging with assets datasets

assets = ['CURRENCYCOM_SILVER', 'SP_SPX', 'TVC_VIX', 'CURRENCYCOM_GOLD', 'TVC_USOIL']

for asset in assets:
    file_str_1D = 'datasets/{}, 1D.csv'.format(asset)
    df = pd.read_csv(file_str_1D)[['time','open', 'close']]
    df[asset + '_mean'] = df[['open', 'close']].mean(axis = 1)
    df = df[['time', asset + '_mean']]
    
    # Convert to timestamp
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['time'] = pd.to_datetime(df['time']).dt.date
    
    # Merge with main df
    merge_df = pd.merge(merge_df, df, on = 'time', how = 'left')

merge_df = merge_df.rename(columns={'CURRENCYCOM_SILVER_mean': 'silver_mean', 'SP_SPX_mean': 'sp500_mean', 'TVC_VIX_mean' : 'vix_mean', 'CURRENCYCOM_GOLD_mean' : 'gold_mean', 'TVC_USOIL_mean' : 'crude_oil'})


In [9]:
# Merging with S2F datasets

df = pd.read_csv('btc.csv')[['date','BlkCnt', 'SplyCur']]
df['total_block'] = df['BlkCnt'].cumsum()
df['btc_s2f'] = df['SplyCur']/(df['SplyCur'].diff()*365)
df['btc_s2f_price'] = e**(-1.84)*(df['btc_s2f']**3.36)
df.rename(columns = {'date' : 'time'}, inplace=True)
df = df[['time', 'btc_s2f', 'btc_s2f_price']]
df['time'] = pd.to_datetime(df['time']).dt.date

merge_df = pd.merge(merge_df, df, on = 'time', how = 'left')

In [11]:
# Merging with NLP results

df = pd.read_csv('merged_with_title.csv')
df.rename(columns = {'date' : 'time', 'compound' : 'nlp_compound', 'Subjectivity' : 'nlp_subjectivity', 'Polarity' : 'nlp_polarity'}, inplace = True)

df['time'] = pd.to_datetime(df['time']).dt.date
df = df.groupby('time', dropna = True).mean()[['nlp_compound', 'nlp_subjectivity', 'nlp_polarity']]
df.reset_index(inplace = True)

merge_df = pd.merge(merge_df, df, on = 'time', how = 'left')

In [13]:
# Merging with Fear & Greed index

df = pd.read_csv('fear_greed.csv')
df.rename(columns = {'fng_value' : 'time' , 'date' : 'sentiment', 'fng_classification' : 'weight'}, inplace=True)
df['time'] = pd.to_datetime(df['time']).dt.date
df = pd.get_dummies(df, columns=['sentiment'])

df.drop('weight', axis = 1, inplace=True)
merge_df = pd.merge(merge_df, df, on = 'time', how = 'left')

In [ ]:
# Drop NA and sort_values
merge_df.sort_values('time').dropna().drop_duplicates().reset_index().drop(columns = 'index', inplace = True, axis = 1)

In [ ]:
merge_df

In [17]:
# Export csv
merge_df.to_csv('aggregated_df_final01oct2021.csv', index = False)

In [ ]:
# Checking the length of raw datasets

exchanges = ['COINBASE', 'BINANCE', 'FTX', 'HUOBI', 'KUCOIN']
others = ['CRYPTOCAP_BTC.D', 'CRYPTOCAP_ETH.D', 'CRYPTOCAP_TOTAL', 'CRYPTOCAP_TOTAL2', 'BCHAIN_MIREV', 'CURRENCYCOM_SILVER', 'SP_SPX', 'TVC_VIX', 'CURRENCYCOM_GOLD', 'TVC_USOIL']

len = 0
count = 0
len_rows = 0
len_col = 0

for exchange in exchanges:
    file_str_1D = 'datasets/{}_BTCUSDT, 1D.csv'.format(exchange)
    df = pd.read_csv(file_str_1D)
    len+= df.shape[0]*df.shape[1]
    len_rows += df.shape[0]
    count += 1
len_col += 15

for other in others:
    file_str_1D = 'datasets/{}, 1D.csv'.format(other)
    df = pd.read_csv(file_str_1D)
    len+= df.shape[0]*df.shape[1]
    len_rows += df.shape[0]
    len_col += df.shape[1]
    count += 1

    
df = pd.read_csv('btc.csv')
len+= df.shape[0]*df.shape[1]
len_rows += df.shape[0]
len_col += df.shape[1]
count += 1

df = pd.read_csv('fear_greed.csv')
len+= df.shape[0]*df.shape[1]
len_rows += df.shape[0]
len_col += df.shape[1]
count += 1

df = pd.read_csv('crypto_news_parsed_2013-2018_40k.csv')
len+= df.shape[0]*df.shape[1]
len_rows += df.shape[0]
len_col += df.shape[1]
count += 1

news_twitter = 24239*8
twitter = 26119 * 8

len += news_twitter + twitter
len_rows += 24239 + 26119
len_col += 8+8
count += 2

print(len, len_rows, len_col, count)

In [34]:
df1 = pd.read_csv('aggregated_df_final01oct2021.csv')

In [35]:
df1.shape

(1488, 30)

In [37]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1488 entries, 0 to 1487
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   time                     1488 non-null   object 
 1   open_x                   1488 non-null   float64
 2   high_x                   1488 non-null   float64
 3   low_x                    1488 non-null   float64
 4   close_x                  1488 non-null   float64
 5   Basis_x                  1469 non-null   float64
 6   Volume_x                 1488 non-null   float64
 7   RSI_x                    1474 non-null   float64
 8   MACD_x                   1463 non-null   float64
 9   OnBalanceVolume_x        1487 non-null   float64
 10  btc_dom                  1488 non-null   float64
 11  eth_dom                  1488 non-null   float64
 12  crypto_cap               1488 non-null   float64
 13  crypto_cap_exc_BTC       1488 non-null   float64
 14  btc_reward              